https://www.everydayhealth.com/conditions/

In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}


In [4]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问百度服务器，获取cookie
    res = requests.get('https://www.everydayhealth.com/conditions/', headers=headers_pc, proxies= randomproxy,)
    # 将cookieJar数据转化为字典，s
    cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy, cookie_dict

# 1、先获取文章链接列表

In [4]:
def get_term_url(url,tag,letter):
    results = []
    headers_pc, randomproxy, cookie_dict = changeparams()
    pbar.set_description(letter)
    try:
        requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接

        r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
#             encoding = cchardet.detect(r.content)['encoding']
#             html = r.content.decode(encoding)
        r.encoding = 'utf-8'
        html = r.text
        with open('D:/Intern_Project/5_everydayhealth/列表页.html', 'w+', encoding='utf-8') as f:
             f.write(html)

        selector = Selector(text=html)
        termUrl_list = selector.xpath(
            "//li[@class='topicslist__topiccolumncont-item']")
        #print(len(termUrl_list))
        for termUrl in termUrl_list:
            if tag == 'health':
                url = 'https://www.everydayhealth.com' + termUrl.xpath('./a/@href').extract()[0]
            if tag == 'drugs':
                url = 'https://www.everydayhealth.com/drugs/' + termUrl.xpath('./a/@href').extract()[0]
            term = termUrl.xpath('./a/text()').extract()[0]
            # print(term)
            tmp = {
                'term': term,
                'url': url
            }
            results.append(tmp)
            # time.sleep()
        pbar.update(1)
    except Exception as e:
        print(e)
    return results

In [5]:
# 获取drugs(多线程)

def task_monitor(task_list, mt_pool, results):
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
                results.extend(result)
                with open('D:/Intern_Project/5_everydayhealth/drugs术语链接列表.json', 'w', encoding='utf-8') as f:
                    f.write(json.dumps(results, indent=2, ensure_ascii=False)) 
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [6]:
letters = [chr(i) for i in range(97, 123)]# 生成26个英文字母的列表

all_craw_tasks = []
results = []

pbar = tqdm(total=len(letters))

with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for letter in letters:
        
        drugs_url = f'https://www.everydayhealth.com/drugs/{letter}'
        tmp_task = mt_pool.submit(get_term_url, drugs_url, 'drugs', letter)
        all_craw_tasks.append(tmp_task)

    # 监测下载任务进展
    task_monitor(all_craw_tasks, mt_pool, results)

    with open('D:/Intern_Project/5_everydayhealth/drugs术语链接列表.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(results, indent=2, ensure_ascii=False))

y:  96%|███████████████████████████████████████████████████████████████████████████▉   | 25/26 [00:45<00:01,  1.05s/it]

# 2、下载每个文章详情页面

In [9]:
# 用来测试哪些页面能简单获取到
# headers_pc, randomproxy = changeparams()
            
# requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
# s = requests.session()
# s.keep_alive = False # 关闭多余连接

# # url = f'https://www.medicinenet.com/diseases_and_conditions/alpha_u.htm'
# url = f'http://www.everydayhealth.com/'
# # r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
# r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, timeout=10)
# #             encoding = cchardet.detect(r.content)['encoding']
# #             html = r.content.decode(encoding)
# r.encoding = 'utf-8'
# html = r.text
# with open('详情页.html', 'w+', encoding='utf-8') as f:
#     f.write(html)


# selector = Selector(text=html)
# term_url_list = selector.xpath("//div[@class='AZ_results']/ul").extract()
# print(len(term_url_list))
# for term_url in term_url_list:
#     try:
#         url = term_url.xpath('./li/a/@href').extract()[0]
#         if url is None or len(url) == 0:
#             print("失败")
#             continue
#         term = term_url.xpath('./li/a/text()').extract()[0]
#         print(url)
#         print(term)
#     except Exception as e:
#         print(e)

In [58]:
# 获取health
health_url = f'https://www.everydayhealth.com/conditions/'
health_results = get_term_url(health_url,'health')
with open('D:/Intern_Project/5_everydayhealth/health术语链接列表.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(health_results, indent=2, ensure_ascii=False))

Abdominal Pain
Ablation
Abortion
ACE Inhibitors
Acetylcholine
Acinetobacter
ACL Tear
Acne
Addiction
Adenoidectomy
ADHD
Agoraphobia
Alcoholism
Allergies
ALS
Alternative Health
Alzheimer's Disease
Amenorrhea
Amino Acids
Aminoglycosides
Amyloidosis
Anal Cancer
Analgesics
Anaphylaxis
Anatomy
Anemia
Anesthesia
Aneurysm
Angina
Angioplasty
Ankylosing Spondylitis
Anorexia
Antacids
Anthrax
Antibiotics
Antidepressants
Antihistamines
Antiphospholipid Syndrome
Antipsychotics
Antisocial Personality Disorder
Anxiety Disorders
Aortic Stenosis
Aplastic anemia
Appendicitis
ARBs
Arm Pain
Aromatase Inhibitors
Arrhythmia
Arthritis
Arthroscopy
ASMR
Asperger's
Aspergillus
Aspiration Pneumonia
Asthma
Atherosclerosis
Athlete's Foot
Athletes foot
Atrial Fibrillation
Autism
Autoimmune Diseases
Back Pain
Bacterial Vaginosis
Barbiturates
Barrett's Esophagus
Beauty Toolbox
Bedbugs
Bedwetting
Benign Prostatic Hyperplasia
Benzodiazepines
Beta Blockers
Binge Eating Disorder
Biopsy
Bipolar Disorder
Bird Flu
Bisphospho

In [6]:
def get_append_article(url):
    
    try:
        requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接

        r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
#             encoding = cchardet.detect(r.content)['encoding']
#             html = r.content.decode(encoding)
        r.encoding = 'utf-8'
        html = r.text

        selector = Selector(text=html)
        articleUrl_list = selector.xpath("//h2/a/@href").extract()
        
        health_append.extend(articleUrl_list)
        
    except Exception as e:
        print(e)

In [9]:
health_append = []
headers_pc, randomproxy, cookie_dict = changeparams()

for i in range(1,40):

    url = f'https://www.everydayhealth.com/news/?page={i}'
    get_append_article(url)
    print(i)
    if i % 5 == 0:
        time.sleep(2)
        headers_pc, randomproxy, cookie_dict = changeparams()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
HTTPSConnectionPool(host='www.everydayhealth.com', port=443): Max retries exceeded with url: /news/?page=38&ie=UTF-8&wd=Python (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027524954850>, 'Connection to www.everydayhealth.com timed out. (connect timeout=10)'))
38
39


In [14]:
health_append[1][31:]

'healthy-living/jabra-hearing-aids-review/'

In [18]:
s = health_append[1][31:].strip('/').replace('/', '--')
print(s)

healthy-living--jabra-hearing-aids-review


In [10]:
with open('D:/Intern_Project/5_everydayhealth/补充文章列表.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(health_append, indent=2, ensure_ascii=False))